In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "../Resources/purchase_data.csv"


In [2]:
# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Check to see if there are any rows with missing data
purchase_data.count()

##   DATA IS CLEAN   ##

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
# rename column
purchase_data = purchase_data.rename(columns={"SN":"User"})
purchase_data.head()

,Purchase ID,User,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
#Total players
count = purchase_data["User"].unique()
print ("Total Number of Players are:", len(count))

Total Number of Players are: 576


In [6]:
#Purchasing Analysis
count1 = purchase_data["Item ID"].unique()
average = round(float(purchase_data["Price"].mean()),2)
count2 = purchase_data["Purchase ID"].unique()
revenue = float(purchase_data["Price"].sum())
analysis_df = pd.DataFrame({
    "Number of Unique Items": [len(count1)],
    "Average Price": [average],
    "Number of Purchases": [len(count2)],
    "Total Revenue": [revenue]
})
analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


In [7]:
user_gender_df = purchase_data[["Gender","User"]].drop_duplicates(subset = "User")

gender_count_df = pd.DataFrame(user_gender_df["Gender"].value_counts(0))
gender_perc_df = pd.DataFrame(user_gender_df["Gender"].value_counts(1))

gender_summary_df = gender_count_df.merge(gender_perc_df, left_index = True, right_index = True)
gender_summary_df.columns = ["Total Count", "Percentage of Players"]
gender_summary_df['Percentage of Players'] = gender_summary_df['Percentage of Players'].map("{:,.2%}".format)
gender_summary_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [9]:
# price_gender_df = purchase_data.groupby("Gender")

# purchase_count = round(price_gender_df["Purchase ID"].count(),0)
# average_count = round(price_gender_df["Price"].mean(),2)
# total_count = round(price_gender_df["Price"].sum(),2)
# average_person = total_count / gender_count_df

# result_df = pd.DataFrame([purchase_count, average_count, total_count, average_person])
# summary2 = result_df.T
# summary2.columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]
# summary2
# Using GroupBy in order to separate the data into fields according to "Gender" values
gender_grouped_purchased_data_df = purchase_data.groupby(["Gender"])

# The object returned is a "GroupBy" object and cannot be viewed normally...
# print(gender_grouped_purchased_data_df)



,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,"$1,967.64",$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35
